# An application of scalar root-finding using root-finding routines

Consider a mechanism comprising four planar rigid rods of lengths $a_1$, $a_2$, $a_3$, and $a_4$. Assume the rod of length $a_1$ is anchored on the horizontal axis with the ends anchored at $(0,0)$ and $(a_1,0)$. Assume further that the remaining rods are connected as illustrated here:

<img src="files/img/rods-01.svg" alt="4-link mechanical linkage" />&nbsp;<img src="files/img/rods-02.svg" alt="4-link mechanical linkage" />

The *Freudenstein equation* 
\begin{equation} \frac{a_{1}}{a_{2}} \cos(\beta) - \frac{a_{1}}{a_{4}} \cos(\alpha) - \cos(\beta-\alpha) =
      - \frac{a_{1}^{2} + a_{2}^{2} - a_{3}^{2} + a_{4}^{2} }{2a_{2}a_{4}} \end{equation}
describes the kinematics of these four rigid planar rods. If one of the angles, say, $\beta$ and all the rod lengths $a_{k}$ ($k=1,\ldots,4$) are prescribed, determining the geometry of the 4-link frame reduces to solving the equation above for the remaining unknown parameter $\alpha$.

To implement this in Octave, let us first create a function ``f_freudenstein`` that can be used in conjunction with a zero-finding routine. Notice the equation is rearranged so that the right-hand side is zero (as would be required for a zero-finding routine).

In [ ]:
function y = f_freudenstein(alpha, beta, A1, A2, A3, A4)
   y = (A1/A2)*cos(beta) - (A1/A4) * cos(alpha) - cos(beta-alpha) + (A1**2+A2**2-A3**2+A4**2)/(2*A2*A4);
end

In [ ]:
f_freudenstein(0.1, 0.25, 11, 13, 8, 10)

To find numerical solutions of the Freudenstein equation, we will use the Octave/Matlab solver `fzero`. Since ``fzero`` expects a single-variable function as an input, one strategy to use ``f_freudenstein`` to find the angle ``alpha`` is to define a temporary, single-variable function using fixed constants ``beta``, ``a1``, ``a2``, ``a3``, and ``a4`` as inputs (this is called *currying* in functional programming).

In [ ]:
a1 = 10; a2 = 13; a3 = 8; a4 = 10;  % Fixing the lengths of the rods
beta = 0.1;  % Fixing one of the angles

In [ ]:
f = @(x) f_freudenstein(x, beta, a1, a2, a3, a4);  % f is a function of a *single* variable x
f(0.5)

With the single-variable function ``f`` thus defined, we can invoke the function ``fzero`` to find the corresponding solution $\alpha$. Notice that we have to provide an initial iterate ``alpha_init`` as well as the function ``f`` as input to ``fzero``; the other input arguments assume default values (we can in principle choose values for these also).

In [ ]:
alpha_init = 1.1;
alpha_sol = fzero(f,alpha_init);
alpha_sol

It is often the case that one wants to solve the same equation with different parameters (i.e., different rod lengths or different angle $\beta$ in this case). Let's *embed* the workflow preceding into a *function*.

In [ ]:
function alpha_sol = solve_freudenstein(alpha_init, beta, A1, A2, A3, A4)
    f = @(x) f_freudenstein(x, beta, A1, A2, A3, A4);
    alpha_sol = fzero(f, alpha_init);
end

Let's just test that the function ``solve_freudenstein`` works as intended.

In [ ]:
a1 = 10; a2 = 13; a3 = 8; a4 = 10;  % Fixing the lengths of the rods
beta = 0.1;  % Fixing one of the angles
alpha_init = pi;
solve_freudenstein(alpha_init, beta, a1, a2, a3, a4)

It worked! So now, let's create an *array* of $\beta$ values. For each of those values of $\beta$, we solve for *two* corresponding $\alpha$ values; let's store those in arrays ``alpha_lo`` and ``alpha_hi``.

In [ ]:
N_beta = 50; % Increase for better resolution
beta_vals = pi * linspace(0, 0.6, N_beta);
alpha_lo = zeros(size(beta_vals));
alpha_hi = alpha_lo;

We need initial values of $\alpha$ to seed an iteration.

In [ ]:
a1 = 10; a2 = 13; a3 = 8; a4 = 10; % Fix rod lengths
% Start by determining the two values of alpha for the first value of beta
beta = beta_vals(1);
alpha_init = -0.2;
alpha_lo(1) = solve_freudenstein(alpha_init, beta, a1, a2, a3, a4);
alpha_init = [0, 2*pi/3];
alpha_hi(1) = solve_freudenstein(alpha_init, beta, a1, a2, a3, a4);

Make a loop now that solves for both values of $\alpha$ for each value of $\beta$ and stores the results.

In [ ]:
for k = 2:N_beta 
    beta = beta_vals(k);
    alpha_init = alpha_lo(k-1); % Use the last point as an initial iterate
    alpha_lo(k) = solve_freudenstein(alpha_init, beta, a1, a2, a3, a4);
    alpha_init = alpha_hi(k-1); % Use the last point as an initial iterate
    alpha_hi(k) = solve_freudenstein(alpha_init, beta, a1, a2, a3, a4);
end

Finally, make a plot to visualize the results.

In [ ]:
plot(beta_vals, alpha_lo, 'r')
hold on
plot(beta_vals, alpha_hi, 'b')
xlabel('beta')
ylabel('alpha')